In [98]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import random
import itertools
import math

In [99]:
import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [112]:
df = pd.read_csv("/Users/ranjani/taxidata/green_tripdata_2019-01.csv")
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2018-12-21 15:17:29,2018-12-21 15:18:57,N,1,264,264,5,0.00,3.0,0.5,0.5,0.00,0.0,NaN,0.3,4.30,2,1,NaN
1,2,2019-01-01 00:10:16,2019-01-01 00:16:32,N,1,97,49,2,0.86,6.0,0.5,0.5,0.00,0.0,NaN,0.3,7.30,2,1,NaN
2,2,2019-01-01 00:27:11,2019-01-01 00:31:38,N,1,49,189,2,0.66,4.5,0.5,0.5,0.00,0.0,NaN,0.3,5.80,1,1,NaN
3,2,2019-01-01 00:46:20,2019-01-01 01:04:54,N,1,189,17,2,2.68,13.5,0.5,0.5,2.96,0.0,NaN,0.3,19.71,1,1,NaN
4,2,2019-01-01 00:19:06,2019-01-01 00:39:43,N,1,82,258,1,4.53,18.0,0.5,0.5,0.00,0.0,NaN,0.3,19.30,2,1,NaN


In [114]:
loc_data = pd.read_csv("/Users/ranjani/taxidata/mygeodata/taxi_zones.csv")
loc_data.head()

,X,Y,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough
0,9.352230e+05,190535.052575,1,0.116357,0.000782,Newark Airport,1,EWR
1,1.032516e+06,167292.493195,2,0.433470,0.004866,Jamaica Bay,2,Queens
2,1.025883e+06,254779.600631,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx
3,9.906188e+05,203105.532318,4,0.043567,0.000112,Alphabet City,4,Manhattan
4,9.314680e+05,139837.478389,5,0.092146,0.000498,Arden Heights,5,Staten Island


In [115]:
df = df.join(loc_data.set_index('LocationID'),on=['PULocationID'])
df.drop(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'borough'], axis=1, inplace=True)
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,X,Y
0,2,2018-12-21 15:17:29,2018-12-21 15:18:57,N,1,264,264,5,0.00,3.0,...,0.00,0.0,NaN,0.3,4.30,2,1,NaN,NaN,NaN
1,2,2019-01-01 00:10:16,2019-01-01 00:16:32,N,1,97,49,2,0.86,6.0,...,0.00,0.0,NaN,0.3,7.30,2,1,NaN,9.910231e+05,190887.289491
2,2,2019-01-01 00:27:11,2019-01-01 00:31:38,N,1,49,189,2,0.66,4.5,...,0.00,0.0,NaN,0.3,5.80,1,1,NaN,9.939749e+05,190123.280316
3,2,2019-01-01 00:46:20,2019-01-01 01:04:54,N,1,189,17,2,2.68,13.5,...,2.96,0.0,NaN,0.3,19.71,1,1,NaN,9.930512e+05,185993.846228
4,2,2019-01-01 00:19:06,2019-01-01 00:39:43,N,1,82,258,1,4.53,18.0,...,0.00,0.0,NaN,0.3,19.30,2,1,NaN,1.019626e+06,208344.701571


In [116]:
df.rename(columns={'X': 'PULocationX', 'Y': 'PULocationY'}, inplace=True)

In [117]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,PULocationX,PULocationY
0,2,2018-12-21 15:17:29,2018-12-21 15:18:57,N,1,264,264,5,0.00,3.0,...,0.00,0.0,NaN,0.3,4.30,2,1,NaN,NaN,NaN
1,2,2019-01-01 00:10:16,2019-01-01 00:16:32,N,1,97,49,2,0.86,6.0,...,0.00,0.0,NaN,0.3,7.30,2,1,NaN,9.910231e+05,190887.289491
2,2,2019-01-01 00:27:11,2019-01-01 00:31:38,N,1,49,189,2,0.66,4.5,...,0.00,0.0,NaN,0.3,5.80,1,1,NaN,9.939749e+05,190123.280316
3,2,2019-01-01 00:46:20,2019-01-01 01:04:54,N,1,189,17,2,2.68,13.5,...,2.96,0.0,NaN,0.3,19.71,1,1,NaN,9.930512e+05,185993.846228
4,2,2019-01-01 00:19:06,2019-01-01 00:39:43,N,1,82,258,1,4.53,18.0,...,0.00,0.0,NaN,0.3,19.30,2,1,NaN,1.019626e+06,208344.701571


In [118]:
df = df.join(loc_data.set_index('LocationID'),on=['DOLocationID'])
df.drop(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'borough'], axis=1, inplace=True)
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,PULocationX,PULocationY,X,Y
0,2,2018-12-21 15:17:29,2018-12-21 15:18:57,N,1,264,264,5,0.00,3.0,...,NaN,0.3,4.30,2,1,NaN,NaN,NaN,NaN,NaN
1,2,2019-01-01 00:10:16,2019-01-01 00:16:32,N,1,97,49,2,0.86,6.0,...,NaN,0.3,7.30,2,1,NaN,9.910231e+05,190887.289491,9.939749e+05,190123.280316
2,2,2019-01-01 00:27:11,2019-01-01 00:31:38,N,1,49,189,2,0.66,4.5,...,NaN,0.3,5.80,1,1,NaN,9.939749e+05,190123.280316,9.930512e+05,185993.846228
3,2,2019-01-01 00:46:20,2019-01-01 01:04:54,N,1,189,17,2,2.68,13.5,...,NaN,0.3,19.71,1,1,NaN,9.930512e+05,185993.846228,9.983427e+05,191392.639499
4,2,2019-01-01 00:19:06,2019-01-01 00:39:43,N,1,82,258,1,4.53,18.0,...,NaN,0.3,19.30,2,1,NaN,1.019626e+06,208344.701571,1.024007e+06,190740.616459


In [127]:
df.rename(columns={'X': 'DOLocationX', 'Y': 'DOLocationY'}, inplace=True)

df.count()

VendorID                 637062
lpep_pickup_datetime     637062
lpep_dropoff_datetime    637062
store_and_fwd_flag       637062
RatecodeID               637062
PULocationID             637062
DOLocationID             637062
passenger_count          637062
trip_distance            637062
fare_amount              637062
extra                    637062
mta_tax                  637062
tip_amount               637062
tolls_amount             637062
ehail_fee                     0
improvement_surcharge    637062
total_amount             637062
payment_type             637062
trip_type                637062
congestion_surcharge      85283
PULocationX              637062
PULocationY              635721
DOLocationX              637062
DOLocationY              634937
dtype: int64

In [120]:
df['PULocationX'] = df['PULocationX'].apply(lambda x: '{:.2f}'.format(x))
df['DOLocationX'] = df['DOLocationX'].apply(lambda x: '{:.2f}'.format(x))
print(df)


        VendorID lpep_pickup_datetime lpep_dropoff_datetime  \
0              2  2018-12-21 15:17:29   2018-12-21 15:18:57   
1              2  2019-01-01 00:10:16   2019-01-01 00:16:32   
2              2  2019-01-01 00:27:11   2019-01-01 00:31:38   
3              2  2019-01-01 00:46:20   2019-01-01 01:04:54   
4              2  2019-01-01 00:19:06   2019-01-01 00:39:43   
5              2  2019-01-01 00:12:35   2019-01-01 00:19:09   
6              2  2019-01-01 00:47:55   2019-01-01 01:00:01   
7              1  2019-01-01 00:12:47   2019-01-01 00:30:50   
8              2  2019-01-01 00:16:23   2019-01-01 00:39:46   
9              2  2019-01-01 00:58:02   2019-01-01 01:19:02   
10             2  2019-01-01 00:37:00   2019-01-01 00:56:42   
11             2  2019-01-01 00:13:48   2019-01-01 00:21:00   
12             2  2019-01-01 00:19:59   2019-01-01 00:45:50   
13             2  2019-01-01 00:57:57   2019-01-01 01:20:10   
14             1  2019-01-01 00:09:02   2019-01-01 00:1

In [121]:
df.isnull().sum()

VendorID                      0
lpep_pickup_datetime          0
lpep_dropoff_datetime         0
store_and_fwd_flag            0
RatecodeID                    0
PULocationID                  0
DOLocationID                  0
passenger_count               0
trip_distance                 0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
ehail_fee                637062
improvement_surcharge         0
total_amount                  0
payment_type                  0
trip_type                     0
congestion_surcharge     551779
PULocationX                   0
PULocationY                1341
DOLocationX                   0
DOLocationY                2125
dtype: int64

In [124]:
nan_rows = df[df['ehail_fee'].isnull()]
#print(nan_rows)

In [125]:
df.dropna(subset=['PULocationY', 'DOLocationY'])

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,PULocationX,PULocationY,DOLocationX,DOLocationY
1,2,2019-01-01 00:10:16,2019-01-01 00:16:32,N,1,97,49,2,0.86,6.0,...,NaN,0.3,7.30,2,1,NaN,991023.10,190887.289491,993974.88,190123.280316
2,2,2019-01-01 00:27:11,2019-01-01 00:31:38,N,1,49,189,2,0.66,4.5,...,NaN,0.3,5.80,1,1,NaN,993974.88,190123.280316,993051.23,185993.846228
3,2,2019-01-01 00:46:20,2019-01-01 01:04:54,N,1,189,17,2,2.68,13.5,...,NaN,0.3,19.71,1,1,NaN,993051.23,185993.846228,998342.74,191392.639499
4,2,2019-01-01 00:19:06,2019-01-01 00:39:43,N,1,82,258,1,4.53,18.0,...,NaN,0.3,19.30,2,1,NaN,1019626.12,208344.701571,1024007.11,190740.616459
5,2,2019-01-01 00:12:35,2019-01-01 00:19:09,N,1,49,17,1,1.05,6.5,...,NaN,0.3,7.80,1,1,NaN,993974.88,190123.280316,998342.74,191392.639499
6,2,2019-01-01 00:47:55,2019-01-01 01:00:01,N,1,255,33,1,3.77,13.5,...,NaN,0.3,14.80,1,1,NaN,996132.62,201170.111116,985545.31,192934.884487
7,1,2019-01-01 00:12:47,2019-01-01 00:30:50,N,1,76,225,1,4.10,16.0,...,NaN,0.3,17.30,1,1,NaN,1017983.41,179276.980156,1003262.22,190226.847965
8,2,2019-01-01 00:16:23,2019-01-01 00:39:46,N,1,25,89,1,7.75,25.5,...,NaN,0.3,26.80,1,1,NaN,988005.47,189064.601119,994603.47,172805.876040
9,2,2019-01-01 00:58:02,2019-01-01 01:19:02,N,1,85,39,1,3.68,15.5,...,NaN,0.3,16.80,1,1,NaN,997512.06,175092.434708,1012066.26,172053.111782
10,2,2019-01-01 00:37:00,2019-01-01 00:56:42,N,1,223,238,1,6.84,22.0,...,NaN,0.3,37.06,1,1,NaN,1010097.76,222807.414929,991777.64,227739.890845
